<a href="https://colab.research.google.com/github/valtjimbo/E-sports/blob/master/Esports_usecase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Planteamiento y carga de datos
1. Partimos de un dataset con una línea por jugador con estadísticas de cada partida.
2. Limpieza y agrupación de variables (variables de jugador que no aportan info sobre el equipo (*share*)
3. Clasificar los equipos para cada grupo de variables (clusters).
4. De todas las clasificaciones para un equipo en un grupo de variables escoger la media y asignarla como definitiva para ese grupo.
5. Dataset con una línea por equipo con sus clusters, el equipo contrario, los clusters del equipo contrario, si fue victoria del local (azul) o no, y una columna con la tendencia en las últimas 5 partidas del local.
 1. *Local* es equipo azul y *Visitante* es equipo rojo
 2. *Local trend 5*: tendencia del local en sus últimas 5 partidas como local
 3. *Visitant trend 5*: tendencia del visitante en sus últimas 5 partidas como visitante
6. Predicciones (Árbol, GradientBoosting, SVM, RandomForest, AdaBoost)



In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
df = pd.read_excel("./dataset/2017-spring-data.xlsx")
df = df.reset_index().drop("index", axis=1)
df.head()

,gameid,url,league,split,date,week,game,patchno,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,k,d,a,teamkills,teamdeaths,doubles,triples,quadras,pentas,fb,fbvictim,fbtime,kpm,okpm,ckpm,fd,fdtime,teamdragkills,oppdragkills,...,opptowerkills,fbaron,fbarontime,teambaronkills,oppbaronkills,dmgtochamps,dmgtochampsperminute,dmgshare,earnedgoldshare,wards,wpm,wardshare,wardkills,wcpm,visionwards,visionwardbuys,visiblewardclearrate,invisiblewardclearrate,totalgold,earnedgpm,goldspent,gspd,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,cssharepost15,csat10,oppcsat10,csdat10,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
0,30267,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 02:10:23,1.1,1,7.01,1,Blue,Top,Lindarang,ROX Tigers,Poppy,Jayce,Rengar,Lee Sin,Varus,Jhin,46.466667,0,1,1,7,13,12,0,0,0,0,0,0,13.7239,0.0215208,0.0215208,0.53802,0,20.058383,0,4,...,11,0.0,32.2174,0.0,2.0,18036.0,388.149211,0.191836,0.19336,31.0,0.667145,0.155779,1.0,0.021521,8,8.0,NaN,NaN,15512.0,205.938594,14600.0,-0.292897,363.0,0.0,0.0,0.0,7.812052,0.220114,82.0,83.0,-1.0,3156.0,3139.0,17.0,4762.0,4900.0,-138.0,4509.0,4744.0,-235.0
1,30267,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 02:10:23,1.1,1,7.01,2,Blue,Jungle,Seonghwan,ROX Tigers,Graves,Jayce,Rengar,Lee Sin,Varus,Jhin,46.466667,0,1,3,6,13,12,0,0,0,0,0,0,13.7239,0.0215208,0.0860832,0.53802,0,20.058383,0,4,...,11,0.0,32.2174,0.0,2.0,16967.0,365.143472,0.180465,0.17473,43.0,0.925395,0.216080,26.0,0.559541,13,15.0,NaN,NaN,14590.0,186.096413,14100.0,-0.118432,103.0,149.0,85.0,64.0,5.423242,0.160342,47.0,39.0,8.0,2708.0,2800.0,-92.0,4359.0,4291.0,68.0,3001.0,2889.0,112.0
2,30267,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 02:10:23,1.1,1,7.01,3,Blue,Middle,Mickey,ROX Tigers,Syndra,Jayce,Rengar,Lee Sin,Varus,Jhin,46.466667,0,5,3,5,13,12,0,0,0,0,0,0,13.7239,0.107604,0.0215208,0.53802,0,20.058383,0,4,...,11,0.0,32.2174,0.0,2.0,25495.0,548.672884,0.271171,0.266871,26.0,0.559541,0.130653,9.0,0.193687,4,4.0,NaN,NaN,19150.0,284.231277,19250.0,-0.179669,462.0,17.0,14.0,3.0,10.308465,0.310247,94.0,92.0,2.0,3460.0,3439.0,21.0,5359.0,5199.0,160.0,4818.0,5160.0,-342.0
3,30267,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 02:10:23,1.1,1,7.01,4,Blue,ADC,Sangyoon,ROX Tigers,Ashe,Jayce,Rengar,Lee Sin,Varus,Jhin,46.466667,0,4,1,6,13,12,0,0,0,0,0,0,13.7239,0.0860832,0.107604,0.53802,0,20.058383,0,4,...,11,0.0,32.2174,0.0,2.0,14534.0,312.783357,0.154587,0.242361,14.0,0.301291,0.070352,8.0,0.172166,3,3.0,NaN,NaN,17937.0,258.126542,16825.0,-0.067924,368.0,42.0,39.0,3.0,8.823529,0.272296,78.0,85.0,-7.0,3005.0,3199.0,-194.0,4747.0,4879.0,-132.0,2895.0,3080.0,-185.0
4,30267,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 02:10:23,1.1,1,7.01,5,Blue,Support,Key,ROX Tigers,Malzahar,Jayce,Rengar,Lee Sin,Varus,Jhin,46.466667,0,2,4,8,13,12,0,0,0,0,1,0,13.7239,0.0430416,0.0215208,0.53802,0,20.058383,0,4,...,11,0.0,32.2174,0.0,2.0,18986.0,408.593974,0.20194,0.122678,85.0,1.829268,0.427136,31.0,0.667145,27,27.0,NaN,NaN,12014.0,130.658824,11625.0,-0.042105,46.0,1.0,1.0,0.0,1.011478,0.037002,4.0,2.0,2.0,2001.0,2182.0,-181.0,3306.0,3228.0,78.0,2681.0,2459.0,222.0


In [3]:
teams = df.loc[df['player'] == 'Team']
teams = teams.reset_index().drop("index", axis=1)
teams.head()

,gameid,url,league,split,date,week,game,patchno,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,k,d,a,teamkills,teamdeaths,doubles,triples,quadras,pentas,fb,fbvictim,fbtime,kpm,okpm,ckpm,fd,fdtime,teamdragkills,oppdragkills,...,opptowerkills,fbaron,fbarontime,teambaronkills,oppbaronkills,dmgtochamps,dmgtochampsperminute,dmgshare,earnedgoldshare,wards,wpm,wardshare,wardkills,wcpm,visionwards,visionwardbuys,visiblewardclearrate,invisiblewardclearrate,totalgold,earnedgpm,goldspent,gspd,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,cssharepost15,csat10,oppcsat10,csdat10,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
0,30267,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 02:10:23,1.1,1,7.01,100,Blue,Team,Team,ROX Tigers,,Jayce,Rengar,Lee Sin,Varus,Jhin,46.466667,0,13,12,32,13,12,,,,,1,0,13.7239,0.27977,0.25825,0.53802,0,20.058383,0,4,...,11,0.0,32.217400,0.0,2.0,94018.0,2023.342898,,,199.0,4.282640,1.0,75.0,1.614060,55,57.0,0.561404,0.330769,79203.0,1065.051650,76400.0,-0.148642,1342.0,209.0,139.0,70.0,33.378766,NaN,305.0,301.0,4.0,14330.0,14759.0,-429.0,22533.0,22497.0,36.0,17904.0,18332.0,-428.0
1,30267,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 02:10:23,1.1,1,7.01,200,Red,Team,Team,KT Rolster,,LeBlanc,Camille,Zyra,Rek'Sai,Kha'zix,46.466667,1,12,13,28,12,13,,,,,0,1,13.7239,0.25825,0.27977,0.53802,1,20.058383,4,0,...,4,1.0,32.217400,2.0,0.0,112397.0,2418.873745,,,187.0,4.024390,1.0,62.0,1.334290,39,40.0,0.559322,0.207143,88307.0,1260.977044,88668.0,0.148642,1228.0,233.0,122.0,111.0,31.441894,NaN,301.0,305.0,-4.0,14759.0,14330.0,429.0,22497.0,22533.0,-36.0,18332.0,17904.0,428.0
2,30279,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 04:01:26,1.1,2,7.01,100,Blue,Team,Team,KT Rolster,,Maokai,Malzahar,Elise,Zyra,Karma,44.016667,1,16,12,37,16,12,,,,,0,1,7.36567,0.363499,0.272624,0.636123,1,11.717217,3,2,...,2,1.0,27.129533,2.0,0.0,92954.0,2111.790988,,,187.0,4.248391,1.0,70.0,1.590307,44,48.0,0.587302,0.226027,86916.0,1333.626657,80245.0,0.094965,1272.0,285.0,139.0,146.0,35.372965,NaN,306.0,294.0,12.0,15474.0,14919.0,555.0,24805.0,23666.0,1139.0,18636.0,18522.0,114.0
3,30279,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 04:01:26,1.1,2,7.01,200,Red,Team,Team,ROX Tigers,,LeBlanc,Camille,Rengar,Rumble,Thresh,44.016667,0,12,16,24,12,16,,,,,1,0,7.36567,0.272624,0.363499,0.636123,0,11.717217,2,3,...,10,0.0,27.129533,0.0,2.0,79424.0,1804.407421,,,209.0,4.748201,1.0,82.0,1.862931,59,65.0,0.690909,0.333333,75044.0,1063.910640,72970.0,-0.094965,1272.0,178.0,119.0,59.0,32.942067,NaN,294.0,306.0,-12.0,14919.0,15474.0,-555.0,23666.0,24805.0,-1139.0,18522.0,18636.0,-114.0
4,30284,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 05:04:07,1.1,1,7.01,100,Blue,Team,Team,Longzhu Gaming,,Cassiopeia,Ryze,Varus,Jayce,Maokai,33.700000,1,22,6,51,22,6,,,,,1,0,7.7318,0.652819,0.178042,0.830861,1,12.458817,3,0,...,1,1.0,28.960867,1.0,0.0,85287.0,2530.771513,,,109.0,3.234421,1.0,47.0,1.394659,29,30.0,0.763158,0.206897,68156.0,1372.569733,63420.0,0.225205,891.0,176.0,93.0,83.0,31.661721,NaN,301.0,294.0,7.0,15335.0,14600.0,735.0,24858.0,22983.0,1875.0,18246.0,18615.0,-369.0


# Variables

- **Gold**: relacionadas con el oro.
    - ["earnedgpm", "goldspent", "gspd", "goldat10", "oppgoldat10", "gdat10", "goldat15", "oppgoldat15", "gdat15"]
- **CS**: relacionadas con el CS (*farmeo*).
    - ["minionkills", "monsterkills", "monsterkillsownjungle", "monsterkillsenemyjungle", "cspm", "csat10", "oppcsat10", "csdat10"]
- **Objs**: relacionadas con los dragones, barones y heraldos.
    - ["teamdragkills",	"oppdragkills",	"elders", 	"oppelders", "teamtowerkills", "opptowerkills", "teambaronkills",	"oppbaronkills"]
- **Wards**: relacionadas con los objetos de visión conocidos como guardianes de visión, en inglés wards.
    - ["wpm", "wcpm"]
- **Damage**: relacionadas con el daño hecho en la partida.
    - ["dmgtochampsperminute"]
- **Early game**: relacionadas con la fase inicial de la partida (*early game*)considerando todas las variables tipo oro, experiencia, etc...
    - ["fb",	"fbvictim",	"fbtime",	"fd",	"fdtime",	"ft",	"fttime",	"firstmidouter",	"firsttothreetowers",	"fbaron",	"fbarontime", "csat10",	"oppcsat10",	"csdat10",	"goldat10",	"oppgoldat10",	"gdat10",	"goldat15",	"oppgoldat15",	"gdat15",	"xpat10",	"oppxpat10",	"xpdat10"]
- **K/D/A**: Variables que indican la cantidad de *kills*, *deaths* y *assists*.
    - ["d",	"a", "kpm", "okpm",	"ckpm"]

In [4]:
# Eliminar filas con duración de partida 0
teams = teams[teams.gamelength != 0]

In [5]:
# Normalización por duración de la partida d las variables que dependen de ella (métrica por minuto)
teams["goldspent_pm"] = teams["goldspent"] / teams["gamelength"]
teams["minionkills_pm"] = teams["minionkills"] / teams["gamelength"]
teams["monsterkills_pm"] = teams["monsterkills"] / teams["gamelength"]
teams["monsterkillsownjungle_pm"] = teams["monsterkillsownjungle"] / teams["gamelength"]
teams["monsterkillsenemyjungle_pm"] = teams["monsterkillsenemyjungle"] / teams["gamelength"]
teams["teamdragkills_pm"] = teams["teamdragkills"] / teams["gamelength"]
teams["oppdragkills_pm"] = teams["oppdragkills"] / teams["gamelength"]
teams["elders_pm"] = teams["elders"] / teams["gamelength"]
teams["oppelders_pm"] = teams["oppelders"] / teams["gamelength"]
teams["teamtowerkills_pm"] = teams["teamtowerkills"] / teams["gamelength"]
teams["opptowerkills_pm"] = teams["opptowerkills"] / teams["gamelength"]
teams["teambaronkills_pm"] = teams["teambaronkills"] / teams["gamelength"]
teams["oppbaronkills_pm"] = teams["oppbaronkills"] / teams["gamelength"]
teams["d_pm"] = teams["d"] / teams["gamelength"]
teams["a_pm"] = teams["a"] / teams["gamelength"]

In [6]:
gold  = teams[["team", "result", "earnedgpm", "goldspent_pm", "gspd", "goldat10", "oppgoldat10", "gdat10", "goldat15", "oppgoldat15", "gdat15"]]
cs    = teams[["team", "result", "minionkills_pm", "monsterkills_pm", "monsterkillsownjungle_pm", "monsterkillsenemyjungle_pm", "cspm", "csat10", "oppcsat10", "csdat10"]]
objs  = teams[["team", "result", "teamdragkills_pm", "oppdragkills_pm", "elders_pm", "oppelders_pm", "teamtowerkills_pm", "opptowerkills_pm", "teambaronkills_pm", "oppbaronkills_pm"]]
wards = teams[["team", "result", "wpm", "wardshare", "wardkills", "wcpm", "visionwards", "visionwardbuys", "visiblewardclearrate", "invisiblewardclearrate"]]
dmg   = teams[["team", "result", "dmgtochampsperminute"]]
eg    = teams[["team", "result", "fb", "fbvictim", "fbtime", "fd", "fdtime", "ft", "fttime", "firstmidouter", "firsttothreetowers", "fbaron", "fbarontime", "csat10", "oppcsat10", "csdat10", "goldat10", "oppgoldat10", "gdat10", "goldat15", "oppgoldat15", "gdat15", "xpat10", "oppxpat10", "xpdat10"]]
kda   = teams[["team", "result", "d_pm", "a_pm", "kpm", "okpm", "ckpm"]]

In [7]:
# Sustitución de nulos por la media
gold .fillna(gold. mean(), inplace=True)
cs   .fillna(cs   .mean(), inplace=True)
objs .fillna(objs .mean(), inplace=True)
wards.fillna(wards.mean(), inplace=True)
dmg  .fillna(dmg  .mean(), inplace=True)
eg   .fillna(eg   .mean(), inplace=True)
kda  .fillna(kda  .mean(), inplace=True)

In [8]:
gold.head()

,team,result,earnedgpm,goldspent_pm,gspd,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15
0,ROX Tigers,0,1065.051650,1644.189383,-0.148642,14330.0,14759.0,-429.0,22533.0,22497.0,36.0
1,KT Rolster,1,1260.977044,1908.206600,0.148642,14759.0,14330.0,429.0,22497.0,22533.0,-36.0
2,KT Rolster,1,1333.626657,1823.059447,0.094965,15474.0,14919.0,555.0,24805.0,23666.0,1139.0
3,ROX Tigers,0,1063.910640,1657.781144,-0.094965,14919.0,15474.0,-555.0,23666.0,24805.0,-1139.0
4,Longzhu Gaming,1,1372.569733,1881.899110,0.225205,15335.0,14600.0,735.0,24858.0,22983.0,1875.0


In [9]:
t_gold  = gold .groupby("team").mean();
t_cs    = cs   .groupby("team").mean();
t_objs  = objs .groupby("team").mean();
t_wards = wards.groupby("team").mean();
t_dmg   = dmg  .groupby("team").mean();
t_eg    = eg   .groupby("team").mean();
t_kda   = kda  .groupby("team").mean();

In [10]:
t_eg.describe()

,result,fdtime,ft,fttime,firstmidouter,firsttothreetowers,fbaron,fbarontime,csat10,oppcsat10,csdat10,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
count,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
mean,0.486957,12.529287,0.493647,12.697702,0.492373,0.490459,0.493095,27.258687,294.342627,294.839039,-0.496412,15001.116334,15017.403696,-16.287361,23620.275726,23664.917200,-44.641475,18332.722339,18349.140771,-16.418431
std,0.178601,0.686883,0.126415,0.804001,0.118426,0.141164,0.124524,1.003544,8.873730,7.928259,7.913169,218.652177,180.462364,298.550675,472.483458,419.825249,726.281954,268.456380,234.043646,288.192060
min,0.071429,11.024510,0.250000,10.702178,0.222222,0.142857,0.160714,24.753600,264.774194,277.363636,-22.142857,14468.740741,14472.904762,-619.071429,22514.888889,22288.523810,-1483.464286,17628.357143,17866.055556,-664.510204
25%,0.358283,12.052899,0.420285,12.236772,0.426292,0.369444,0.422484,26.955423,291.281915,291.685897,-4.125855,14864.767857,14908.383929,-185.660227,23277.136161,23447.817568,-599.304715,18213.199405,18196.222297,-148.969787
50%,0.485667,12.530633,0.500000,12.721886,0.500000,0.500000,0.500000,27.217221,295.606122,295.547959,0.000000,15033.921429,15033.921429,0.000000,23678.427551,23678.427551,0.000000,18373.923980,18373.923980,0.000000
75%,0.625275,12.882227,0.553030,12.998139,0.567460,0.560764,0.555366,27.610994,300.407751,298.095238,4.460714,15109.160369,15163.116477,146.471429,24016.127381,23867.727639,426.459152,18516.495726,18447.703510,156.083333
max,0.931034,14.726014,0.931034,14.635404,0.724138,0.793103,0.827586,30.102724,310.756098,317.088889,16.878049,15440.840909,15343.571429,732.931034,24550.068966,24702.107143,2031.793103,18786.103448,18899.333333,647.888889


# Hacemos los clusters para "*resumir*" la info del tipo de juego

In [11]:
from sklearn.cluster import KMeans
import numpy as np

In [12]:
gold_  = gold .drop(["result", "team"], axis=1)
cs_    = cs   .drop(["result", "team"], axis=1)
objs_  = objs .drop(["result", "team"], axis=1)
wards_ = wards.drop(["result", "team"], axis=1)
dmg_   = dmg  .drop(["result", "team"], axis=1)
eg_    = eg   .drop(["result", "team"], axis=1)
kda_   = kda  .drop(["result", "team"], axis=1)

In [13]:
t_gold["kmeans"] = KMeans(n_clusters=5, random_state=170).fit_predict (t_gold)
t_cs["kmeans"] = KMeans(n_clusters=5, random_state=170).fit_predict   (t_cs)
t_objs["kmeans"] = KMeans(n_clusters=8, random_state=170).fit_predict (t_objs)
t_wards["kmeans"] = KMeans(n_clusters=5, random_state=170).fit_predict(t_wards)
t_dmg["kmeans"] = KMeans(n_clusters=5, random_state=170).fit_predict  (t_dmg)
t_eg["kmeans"] = KMeans(n_clusters=5, random_state=170).fit_predict   (t_eg)
t_kda["kmeans"] = KMeans(n_clusters=5, random_state=170).fit_predict  (t_kda)

In [14]:
t_gold.head()

,result,earnedgpm,goldspent_pm,gspd,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,kmeans
team,,,,,,,,,,,
AHQ e-Sports Club,0.742857,1225.162598,1673.173881,0.072195,15074.171429,14896.142857,178.028571,23918.600000,23226.171429,692.428571,2
Afreeca Freecs,0.520833,1157.865131,1667.465343,0.011693,14949.791667,15228.916667,-279.125000,23276.062500,23630.187500,-354.125000,4
BBQ Olivers,0.363636,1064.093848,1600.059539,-0.056041,14855.863636,15257.522727,-401.659091,23044.727273,23847.886364,-803.159091,1
CNB e-Sports Club,0.428571,1049.787571,1573.099817,-0.050064,14582.357143,14658.642857,-76.285714,22600.214286,23324.785714,-724.571429,1
Cloud9,0.692308,1185.465707,1651.688422,0.016541,14986.153846,15132.269231,-146.115385,23664.711538,23804.826923,-140.115385,4


In [15]:
# Almacenamos a qué agrupación pertenece cada equipo por cada grupo de variables
df_features = pd.DataFrame()

df_features["team"] =  t_gold           .index
df_features["gold"] =  t_gold ["kmeans"].values
df_features["cs"] =    t_cs   ["kmeans"].values
df_features["objs"] =  t_objs ["kmeans"].values
df_features["wards"] = t_wards["kmeans"].values
df_features["dmg"] =   t_dmg  ["kmeans"].values
df_features["kda"] =   t_kda  ["kmeans"].values
df_features["eg"] =    t_eg   ["kmeans"].values

In [16]:
df_features

,team,gold,cs,objs,wards,dmg,kda,eg
0,AHQ e-Sports Club,2,2,7,3,0,2,3
1,Afreeca Freecs,4,1,3,3,1,0,1
2,BBQ Olivers,1,1,6,2,1,1,0
3,CNB e-Sports Club,1,3,0,3,0,0,0
4,Cloud9,4,2,1,2,4,2,1
...,...,...,...,...,...,...,...,...
61,Vici Gaming,4,0,6,2,0,1,1
62,Vitality,4,0,2,0,1,1,1
63,Wayi Spider,1,1,5,0,3,1,0
64,eXtreme Gamers,1,1,2,0,3,1,0


# Local | Visitante | Ganó el local? (1 o 0) | Ganó el Visitante? (0 o 1) | indice de la partida

In [17]:
# DatasetPartimos del dataset inicial de equipos
# Hay partidas con nulo en "date", por tanto, el dataframe inicial esta ordenado, añadimos una columna ("ind") que será el indice
teams = teams.reset_index()
teams.rename(columns={'index':'ind'}, inplace=True)
teams.head()

,ind,gameid,url,league,split,date,week,game,patchno,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,k,d,a,teamkills,teamdeaths,doubles,triples,quadras,pentas,fb,fbvictim,fbtime,kpm,okpm,ckpm,fd,fdtime,teamdragkills,...,visionwardbuys,visiblewardclearrate,invisiblewardclearrate,totalgold,earnedgpm,goldspent,gspd,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,cssharepost15,csat10,oppcsat10,csdat10,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10,goldspent_pm,minionkills_pm,monsterkills_pm,monsterkillsownjungle_pm,monsterkillsenemyjungle_pm,teamdragkills_pm,oppdragkills_pm,elders_pm,oppelders_pm,teamtowerkills_pm,opptowerkills_pm,teambaronkills_pm,oppbaronkills_pm,d_pm,a_pm
0,0,30267,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 02:10:23,1.1,1,7.01,100,Blue,Team,Team,ROX Tigers,,Jayce,Rengar,Lee Sin,Varus,Jhin,46.466667,0,13,12,32,13,12,,,,,1,0,13.7239,0.27977,0.25825,0.53802,0,20.058383,0,...,57.0,0.561404,0.330769,79203.0,1065.051650,76400.0,-0.148642,1342.0,209.0,139.0,70.0,33.378766,NaN,305.0,301.0,4.0,14330.0,14759.0,-429.0,22533.0,22497.0,36.0,17904.0,18332.0,-428.0,1644.189383,28.880918,4.497848,2.991392,1.506456,0.000000,0.086083,0.000000,0.021521,0.086083,0.236729,0.000000,0.043042,0.258250,0.688666
1,1,30267,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 02:10:23,1.1,1,7.01,200,Red,Team,Team,KT Rolster,,LeBlanc,Camille,Zyra,Rek'Sai,Kha'zix,46.466667,1,12,13,28,12,13,,,,,0,1,13.7239,0.25825,0.27977,0.53802,1,20.058383,4,...,40.0,0.559322,0.207143,88307.0,1260.977044,88668.0,0.148642,1228.0,233.0,122.0,111.0,31.441894,NaN,301.0,305.0,-4.0,14759.0,14330.0,429.0,22497.0,22533.0,-36.0,18332.0,17904.0,428.0,1908.206600,26.427547,5.014347,2.625538,2.388809,0.086083,0.000000,0.021521,0.000000,0.236729,0.086083,0.043042,0.000000,0.279770,0.602582
2,2,30279,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 04:01:26,1.1,2,7.01,100,Blue,Team,Team,KT Rolster,,Maokai,Malzahar,Elise,Zyra,Karma,44.016667,1,16,12,37,16,12,,,,,0,1,7.36567,0.363499,0.272624,0.636123,1,11.717217,3,...,48.0,0.587302,0.226027,86916.0,1333.626657,80245.0,0.094965,1272.0,285.0,139.0,146.0,35.372965,NaN,306.0,294.0,12.0,15474.0,14919.0,555.0,24805.0,23666.0,1139.0,18636.0,18522.0,114.0,1823.059447,28.898145,6.474820,3.157895,3.316925,0.068156,0.045437,0.000000,0.022719,0.227187,0.045437,0.045437,0.000000,0.272624,0.840591
3,3,30279,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 04:01:26,1.1,2,7.01,200,Red,Team,Team,ROX Tigers,,LeBlanc,Camille,Rengar,Rumble,Thresh,44.016667,0,12,16,24,12,16,,,,,1,0,7.36567,0.272624,0.363499,0.636123,0,11.717217,2,...,65.0,0.690909,0.333333,75044.0,1063.910640,72970.0,-0.094965,1272.0,178.0,119.0,59.0,32.942067,NaN,294.0,306.0,-12.0,14919.0,15474.0,-555.0,23666.0,24805.0,-1139.0,18522.0,18636.0,-114.0,1657.781144,28.898145,4.043923,2.703521,1.340401,0.045437,0.068156,0.022719,0.000000,0.045437,0.227187,0.000000,0.045437,0.363499,0.545248
4,4,30284,http://matchhistory.na.leagueoflegends.com/en/...,LCK,2017-1,2017-01-17 05:04:07,1.1,1,7.01,100,Blue,Team,Team,Longzhu Gaming,,Cassiopeia,Ryze,Varus,Jayce,Maokai,33.700000,1,22,6,51,22,6,,,,,1,0,7.7318,0.652819,0.178042,0.830861,1,12.458817,3,...,30.0,0.763158,0.206897,68156.0,1372.569733,63420.0,0.225205,891.0,176.0,93.0,83.0,31.661721,NaN,301.0,294.0,7.0,15335.0,14600.0,735.0,24858.0,22983.0,1875.0,18246.0,18615.0,-369.0,1881.899110,26.439169,5.222552,2.759644,2.462908,0.089021,0.000000,0.000000,0.000000,0.326409,0.029674,0.029674,0.000000,0.178042,1.513353


In [18]:
# Dataframe (df_match) con info de las partidas jugadas por los equipos
# Local | Visitante | Ganó el local? (1 o 0) | Ganó el Visitante? (0 o 1) | indice de la partida
# La columna gano el visitante se necesitará para calcular las tendencias del equipo visitante más adelante

df_match = pd.DataFrame()
df_match_loc = pd.DataFrame()
df_match_vis = pd.DataFrame()
df_match_date = pd.DataFrame()

# En el dataframe teams tenemos la informacion de las partidas segun equipos. 
# El formato es:
# Fila 0 -> Equipo Local - indice de la partida 1 - ...informacion de la partida 1...
# Fila 1 -> Equipo Visitante - indice de la partida 1 - ...informacion de la partida 1...
# Fila 2 -> Equipo Local - indice de la partida 2 - ...informacion de la partida 2...
# Fila 3 -> Equipo Visitante - indice de la partida 2 - ...informacion de la partida 2...

# por tanto para los locales recorremos el dataframe de dos en dos desde el 0
df_match_loc = teams[::2]["team"]
df_match_loc = df_match_loc.reset_index().drop("index", axis=1)

# para los visitantes recorremos el dataframe de dos en dos desde el 1
df_match_vis = teams[1::2]["team"]
df_match_vis = df_match_vis.reset_index().drop("index", axis=1)

# Cogemos el resultado de la partida para saber quien ganó
df_match_res = teams[::2]["result"]
df_match_res = df_match_res.reset_index().drop("index", axis=1)

# Lo trasladamos a df_match
df_match['ind'] = teams['ind']
df_match["Local"] = df_match_loc
df_match["Visitant"] = df_match_vis
df_match["Local Win"] = df_match_res

# Calculamos Visitant Win a partir de Local Win
df_match['Visitant Win'] = df_match['Local Win'].apply(lambda x: (x+1)%2)

df_match = df_match.dropna()
df_match

,ind,Local,Visitant,Local Win,Visitant Win
0,0,ROX Tigers,KT Rolster,0.0,1.0
1,1,KT Rolster,ROX Tigers,1.0,0.0
2,2,Longzhu Gaming,Samsung Galaxy,1.0,0.0
3,3,Samsung Galaxy,Longzhu Gaming,1.0,0.0
4,4,Longzhu Gaming,Samsung Galaxy,0.0,1.0
...,...,...,...,...,...
1233,1233,SuperMassive,Galatasaray Esports,0.0,1.0
1234,1234,Crew e-Sports Club,SuperMassive,0.0,1.0
1235,1235,SuperMassive,Crew e-Sports Club,0.0,1.0
1236,1236,Crew e-Sports Club,SuperMassive,0.0,1.0


# Tendencias de últimas 5 partidas

In [19]:
from collections import defaultdict as dd

dfsize = df_match.shape[0]

# LOCAL
df_localwin = pd.DataFrame()
df_localwin["Local"] = df_match["Local"] 
df_localwin["Local Win"] = df_match["Local Win"] 
df_localwin["5 trend loc"] = 0
df_localwin["num"] = 0
df_localwin["wins"] = 0
df_localwin["ind"] = df_match["ind"] 
df_localwin = df_localwin.reset_index().drop("index", axis=1)

# VISITANTE
df_viswin = pd.DataFrame()
df_viswin["Visitant"] = df_match["Visitant"] 
df_viswin["Visitant Win"] = df_match["Visitant Win"] 
df_viswin["5 trend vis"] = 0
df_viswin["num"] = 0
df_viswin["wins"] = 0
df_viswin["ind"] = df_match["ind"] 
df_viswin = df_viswin.reset_index().drop("index", axis=1)

# Diccionarios
localwindd = dd(int)
numlocaldd = dd(int)
winantloc = dd(int)
index_auxloc = 0

viswindd = dd(int)
numvisdd = dd(int)
winantvis = dd(int)
index_auxvis = 0

listwinsloc = [0]*dfsize
listnumloc = [0]*dfsize

listwinsvis = [0]*dfsize
listnumvis = [0]*dfsize

In [20]:
# Número de victorias y número de partidas para el local
for match in df_localwin.iterrows():
        team = match[1][0]
        localwindd[team] += match[1][1]
        numlocaldd[team] += 1
        listwinsloc[index_auxloc] = localwindd[team]
        listnumloc[index_auxloc] = numlocaldd[team]
        index_auxloc +=1

df_localwin['num'] = listnumloc
df_localwin['wins'] = listwinsloc
df_localwin = df_localwin.sort_values(["Local", "num"])
df_localwin = df_localwin.reset_index().drop("index", axis=1)

In [21]:
# Número de victorias y numero de partidas para el visitante
for match in df_viswin.iterrows():
        team = match[1][0]
        viswindd[team] += match[1][1]
        numvisdd[team] += 1
        listwinsvis[index_auxvis] = viswindd[team]
        listnumvis[index_auxvis] = numvisdd[team]
        index_auxvis +=1

df_viswin['num'] = listnumvis
df_viswin['wins'] = listwinsvis
df_viswin = df_viswin.sort_values(["Visitant", "num"])
df_viswin = df_viswin.reset_index().drop("index", axis=1)


In [22]:
# Local
from tqdm import tqdm 

listantloc = [0]*dfsize
listantgloballoc = [0]*dfsize
listactloc = [0]*dfsize
listtrendloc = [0]*dfsize
listnwinsloc = [0]*dfsize
listglobaltrendloc = [0]*dfsize

listactloc = df_localwin['Local Win']
listnwinsloc = df_localwin['wins']

control = 0
for i in tqdm(range(0,dfsize)):
    j=0
    team_aux = df_localwin["Local"][control]

    while team_aux == df_localwin['Local'][control] and team_aux != "stop": 
        if control == dfsize-1:
            team_aux = "stop"
        else:
            if (j > 0 and j <= 4):
                listantloc[control] = listantloc[control-1] + listactloc[control-1]
                listtrendloc[control] = listantloc[control] / j
                
            elif j > 4:
                listantloc[control] = listactloc[control-1] + listactloc[control-2] + listactloc[control-3] + listactloc[control-4] + listactloc[control-5]
                listtrendloc[control] = listantloc[control] / 5            
                        
            team_aux = df_localwin['Local'][control] 
            control += 1
            j += 1

# Caso final
listantloc[dfsize-1] = listactloc[dfsize-2] + listactloc[dfsize-3] + listactloc[dfsize-4] + listactloc[dfsize-5] + listactloc[dfsize-6]
listtrendloc[dfsize-1] = listantloc[dfsize-1] / 5

df_localwin['wins_ant'] = listantloc
df_localwin['5 trend loc'] = listtrendloc

# Tenemos:
# Equipo local | Gano el local? | tendencia de las ultimas 5 partidas del equipo como local
# | numero de partidas que jugo de local | numero de victorias que logro como local | indice de la partida
# | wins_ant: hasta llegar a esa partida, cuantas habia ganado en las ultimas 5 como local?

100%|██████████| 1238/1238 [00:00<00:00, 7708.38it/s]


In [23]:
df_localwin

,Local,Local Win,5 trend loc,num,wins,ind,wins_ant
0,AHQ e-Sports Club,1.0,0.00,1,1.0,356,0.0
1,AHQ e-Sports Club,1.0,1.00,2,2.0,358,1.0
2,AHQ e-Sports Club,1.0,1.00,3,3.0,363,2.0
3,AHQ e-Sports Club,0.0,1.00,4,3.0,677,3.0
4,AHQ e-Sports Club,1.0,0.75,5,4.0,818,3.0
...,...,...,...,...,...,...,...
1233,paiN Gaming,1.0,0.80,6,5.0,586,4.0
1234,paiN Gaming,1.0,0.80,7,6.0,718,4.0
1235,paiN Gaming,0.0,0.80,8,6.0,940,4.0
1236,paiN Gaming,1.0,0.60,9,7.0,941,3.0


In [24]:
# Visitante

listantvis = [0]*dfsize
listantglobalvis = [0]*dfsize
listactvis = [0]*dfsize
listtrendvis = [0]*dfsize
listnwinsvis = [0]*dfsize
listglobaltrendvis = [0]*dfsize

listactvis = df_viswin['Visitant Win']
listnwinsvis = df_viswin['wins']

control = 0
for i in tqdm(range(0,dfsize)):
    j=0
    team_aux = df_viswin["Visitant"][control]
    
    while team_aux == df_viswin['Visitant'][control] and team_aux != "stop": 
        if control == dfsize-1:
            team_aux = "stop"
        else:
            if (j > 0 and j <= 4):
                listantvis[control] = listantvis[control-1] + listactvis[control-1]
                listtrendvis[control] = listantvis[control] / j

            elif j > 4:
                listantvis[control] = listactvis[control-1] + listactvis[control-2] + listactvis[control-3] + listactvis[control-4]+ listactvis[control-5]
                listtrendvis[control] = listantvis[control] / 5
            
            team_aux = df_viswin['Visitant'][control] 
            control += 1
            j += 1

# Caso final
listantvis[dfsize-1] = listactvis[dfsize-2] + listactvis[dfsize-3] + listactvis[dfsize-4] + listactvis[dfsize-5] + listactvis[dfsize-6]
listtrendvis[dfsize-1] = listantvis[dfsize-1] / 5

df_viswin['wins_ant'] = listantvis
df_viswin['5 trend vis'] = listtrendvis

# Tenemos:
# Equipo visitante | Gano el visitante? | tendencia de las ultimas 5 partidas del equipo como visitante
# | numero de partidas que jugo de visitante | numero de victorias que logro como visitante | indice de la partida
# | wins_ant: hasta llegar a esa partida, cuantas habia ganado en las ultimas 5 como visitante?

100%|██████████| 1238/1238 [00:00<00:00, 6414.85it/s]


In [25]:
df_viswin

,Visitant,Visitant Win,5 trend vis,num,wins,ind,wins_ant
0,AHQ e-Sports Club,1.0,0.000000,1,1.0,357,0.0
1,AHQ e-Sports Club,1.0,1.000000,2,2.0,359,1.0
2,AHQ e-Sports Club,1.0,1.000000,3,3.0,362,2.0
3,AHQ e-Sports Club,1.0,1.000000,4,4.0,553,3.0
4,AHQ e-Sports Club,1.0,1.000000,5,5.0,554,4.0
...,...,...,...,...,...,...,...
1233,paiN Gaming,0.0,0.666667,4,2.0,291,2.0
1234,paiN Gaming,1.0,0.500000,5,3.0,422,2.0
1235,paiN Gaming,0.0,0.600000,6,3.0,587,3.0
1236,paiN Gaming,1.0,0.400000,7,4.0,719,2.0


In [26]:
# Nos quedamos con las columnas que nos interesan para local

df_localwin.drop('wins', axis=1, inplace=True)
df_localwin.drop('Local Win', axis=1, inplace=True)

df_localwin = df_localwin.sort_values(["ind"])
df_localwin = df_localwin.reset_index().drop("index", axis=1)

In [27]:
# Nos quedamos con las columnas que nos interesan para visitante

df_viswin.drop('wins', axis=1, inplace=True)
df_viswin.drop('Visitant Win', axis=1, inplace=True)

df_viswin = df_viswin.sort_values(["ind"])
df_viswin = df_viswin.reset_index().drop("index", axis=1)

In [28]:
# Añadimos a df_match las columnas que nos interesan
df_match.loc[:,"5 trend loc"] = df_localwin['5 trend loc']
df_match.loc[:,"5 trend vis"] = df_viswin['5 trend vis']

# numero de partidas lo necesitaremos porque vamos a eliminar las primeras 5 partidas de cada equipo como
# local y como visitante, ya que la tendencia calculada en esas partidas puede confundir al modelo,
# ya que es un dato engañoso comparado con el de las tendencias del resto de partidas
df_match.loc[:,"num loc"] = df_localwin['num']
df_match.loc[:,"num vis"] = df_viswin['num']

# df_match = df_match.sort_values(["ind", "Local", "Visitant"])

In [29]:
# también añadimos del dataframe features, los clusters de las caracteristicas
df_match = df_match.merge(df_features, left_on='Local', right_on='team', how='inner')
df_match.drop('team', axis=1, inplace= True)

df_match = df_match.merge(df_features, left_on='Visitant', right_on='team', how='inner')
df_match.drop('team', axis=1, inplace= True)

In [30]:
df_match = df_match.sort_values(["ind"])
df_match = df_match.reset_index().drop("index", axis=1)

In [31]:
# Eliminamos la informacion de las 5 primeras partidas de cada equipo como local y las 5 primeras como visitante, ya 
# que la tendencia calculada hasta ese punto puede dar un dato engañoso
df_match = df_match[df_match["num loc"] > 5]
df_match = df_match[df_match["num vis"] > 5]
df_match = df_match.reset_index().drop("index", axis=1)

In [32]:
# eliminamos el indice y la columna "visitant win" que ya no necesitamos
df_match.drop('ind', axis=1, inplace=True)
df_match.drop('Visitant Win', axis=1, inplace=True)
df_match.drop('num loc', axis=1, inplace=True)
df_match.drop('num vis', axis=1, inplace=True)

# Tenemos: 
# Local / Visitante / Victoria Local o no / ratio de victorias del local en las ultimas 5 partidas 
# / ratio de victorias del visitante en las ultimas 5 partidas
df_match

,Local,Visitant,Local Win,5 trend loc,5 trend vis,gold_x,cs_x,objs_x,wards_x,dmg_x,kda_x,eg_x,gold_y,cs_y,objs_y,wards_y,dmg_y,kda_y,eg_y
0,Echo Fox,Flyquest,1.0,0.6,0.6,2,0,6,0,0,4,4,2,0,0,1,0,4,1
1,EnVyUs,Team Liquid,1.0,0.2,0.2,2,2,6,3,0,1,4,1,0,6,1,0,4,0
2,Roccat,Fnatic,0.0,0.4,0.4,1,1,0,3,0,0,0,2,2,3,2,0,0,4
3,Team SoloMid,Counter Logic Gaming,1.0,0.8,0.2,2,2,1,1,0,2,4,1,3,0,0,4,4,0
4,Counter Logic Gaming,Team SoloMid,0.0,0.6,0.2,1,3,0,0,4,4,0,2,2,1,1,0,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,SuperMassive,Galatasaray Esports,0.0,1.0,0.0,3,4,7,1,2,3,2,1,3,2,1,4,4,0
868,Crew e-Sports Club,SuperMassive,0.0,0.6,0.6,2,4,3,1,2,3,4,3,4,7,1,2,3,2
869,SuperMassive,Crew e-Sports Club,0.0,0.8,0.2,3,4,7,1,2,3,2,2,4,3,1,2,3,4
870,Crew e-Sports Club,SuperMassive,0.0,0.4,0.6,2,4,3,1,2,3,4,3,4,7,1,2,3,2


# Discretizamos los nombres de equipos

In [33]:
# Discretizamos los nombres de los equipos, y almacenamos la informacion en un nuevo dataframe df_stats, para probar el modelo
from sklearn import preprocessing

local_le = preprocessing.LabelEncoder()
visitante_le = preprocessing.LabelEncoder()

local_le.fit(df_match["Local"].values);
visitante_le.fit(df_match["Visitant"].values);

df_stats = df_match

df_stats["Local"] = local_le.transform(df_stats["Local"])
df_stats["Visitant"] = visitante_le.transform(df_stats["Visitant"])

In [34]:
df_stats

,Local,Visitant,Local Win,5 trend loc,5 trend vis,gold_x,cs_x,objs_x,wards_x,dmg_x,kda_x,eg_x,gold_y,cs_y,objs_y,wards_y,dmg_y,kda_y,eg_y
0,9,15,1.0,0.6,0.6,2,0,6,0,0,4,4,2,0,0,1,0,4,1
1,11,57,1.0,0.2,0.2,2,2,6,3,0,1,4,1,0,6,1,0,4,0
2,49,16,0.0,0.4,0.4,1,1,0,3,0,0,0,2,2,3,2,0,0,4
3,58,5,1.0,0.8,0.2,2,2,1,1,0,2,4,1,3,0,0,4,4,0
4,5,58,0.0,0.6,0.2,1,3,0,0,4,4,0,2,2,1,1,0,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,55,19,0.0,1.0,0.0,3,4,7,1,2,3,2,1,3,2,1,4,4,0
868,6,55,0.0,0.6,0.6,2,4,3,1,2,3,4,3,4,7,1,2,3,2
869,55,6,0.0,0.8,0.2,3,4,7,1,2,3,2,2,4,3,1,2,3,4
870,6,55,0.0,0.4,0.6,2,4,3,1,2,3,4,3,4,7,1,2,3,2


In [35]:
# Cuenta de partidos que gana el local y el visitante
size0 = df_stats.shape[0]
ganoBlue = 0
ganoRed = 0
for v in tqdm(range(0,size0)):
    if df_stats["Local Win"][v] == 1:
        ganoBlue += 1
    else:
        ganoRed +=1

print("")    
print("Datos totales: " + str(size0))
print("Ganó Blue: " + str(ganoBlue))
print("Ganó Red: " + str(ganoRed))

100%|██████████| 872/872 [00:00<00:00, 41947.85it/s]


Datos totales: 872
Ganó Blue: 494
Ganó Red: 378


# Modelos de predicción

Usamos validación cruzada para evaluar los modelos:

https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation

In [36]:
# Modelos de predicción
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = df_stats.drop("Local Win", axis=1)
x = X.values
y = df_stats["Local Win"].values

##### Arbol de decision

In [37]:
# Árbol de decisión
from sklearn import tree

clasificador = tree.DecisionTreeClassifier()

# Utilizamos cross-validation
from sklearn.model_selection import cross_val_score

scores1 = cross_val_score(clasificador, X, y, cv=5)
print(scores1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores1.mean(), scores1.std() * 2))

[0.58857143 0.54857143 0.56896552 0.62643678 0.48850575]
Accuracy: 0.56 (+/- 0.09)


##### Gradient boosting

In [38]:
# Gradient boosting Classifier
from sklearn import ensemble

clasificador2 = ensemble.GradientBoostingClassifier()

# Utilizamos cross-validation
scores2 = cross_val_score(clasificador2, X, y, cv=5)
print(scores2)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores2.mean(), scores2.std() * 2))

[0.60571429 0.73142857 0.63793103 0.68390805 0.58045977]
Accuracy: 0.65 (+/- 0.11)


##### SVM

In [39]:
# SVM
from sklearn import svm

clasificador3 = svm.SVC(kernel='linear')

# Utilizamos cross-validation
scores3 = cross_val_score(clasificador3, X, y, cv=5)
print(scores3)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores3.mean(), scores3.std() * 2))

[0.6        0.64571429 0.5862069  0.54597701 0.56896552]
Accuracy: 0.59 (+/- 0.07)


##### Random Forest

In [40]:
# RandomForest

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

clasificador4 = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)

# Utilizamos cross-validation
scores4 = cross_val_score(clasificador4, X, y, cv=5)
print(scores4)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores4.mean(), scores4.std() * 2))

[0.66285714 0.64       0.62643678 0.64367816 0.61494253]
Accuracy: 0.64 (+/- 0.03)


##### AdaBoost

In [41]:
#AdaBoost

clasificador5 = AdaBoostClassifier()

# Utilizamos cross-validation
scores5 = cross_val_score(clasificador5, X, y, cv=5)
print(scores5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores5.mean(), scores5.std() * 2))

[0.60571429 0.72       0.62068966 0.68390805 0.59195402]
Accuracy: 0.64 (+/- 0.10)
